# Gradio Demo

Code creates a Gradio demo that runs locally, where user input is saved to a local JSON file. The demo can also be deployed to Hugging Face Spaces, but when deployed, it requires a connected database for storing user inputs.

In [1]:
!pip -q install gradio -q
!python -m pip install "pymongo[srv]"==3.6 -q

In [1]:
import numpy as np
import pandas as pd
import gradio as gr
import os
from PIL import Image
import json
import random
import re

/Users/piadonabauer/PycharmProjects/gradio/enviro/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
instruction_beginning = """
## 🔍 Evaluation of AI Quality

\n**Background**

\nIn this task, you will evaluate the quality of image edits based on a text prompt and a masked area regarding three aspects: **Prompt-Image Alignment**, **Visual Quality** and **Consistency**.
Each aspect should be rated on a scale from 1 to 10, where 1 indicates 'very poor' and 10 represents 'excellent'.

Please ensure you have read the detailed instructions provided in this [document](https://www.canva.com/design/DAGP0UTTygI/rYkYZtLUipuKbXPbRcj9kQ/edit?utm_content=DAGP0UTTygI&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton) before starting the labeling process.

\n**Labeling**

\nPlease enter a nickname to avoid repeating triplets. Make sure to remember this nickname for future sessions. If you choose not to enter a nickname, your ratings will not be saved.
"""

alignment_info = """
How well does the edited area align with the text prompt? (e.g. numbers, colors, and objects)

"""

quality_info = """
How realistic and aesthetically pleasing is the edited area? (e.g. color realism and overall aesthetics)
"""

consistency_info = """
How seamlessly does the edit integrate with the rest of the original image? (e.g. consistency in style, lighting, logic, and spatial coherence)
"""

overall_info = """
How do you perceive and like the edit as a whole, how well does it meet your expectations and complements the original image?
"""

Download dev file: [link from HuggingFace](https://buckeyemailosu-my.sharepoint.com/personal/zhang_13253_buckeyemail_osu_edu/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fzhang%5F13253%5Fbuckeyemail%5Fosu%5Fedu%2FDocuments%2FDatasets%2FMagicBrush%2FfinalRelease%2Fdev%2Ezip&parent=%2Fpersonal%2Fzhang%5F13253%5Fbuckeyemail%5Fosu%5Fedu%2FDocuments%2FDatasets%2FMagicBrush%2FfinalRelease&ga=1).

In [1]:
directory_path = "./dev/images"

In [3]:
# load image as array
def get_img_array(directory_path, filename):
    image_path = os.path.join(directory_path, filename)
    image = Image.open(image_path)
    image_array = np.array(image)
    return image_array

# generator to load image triplets with corresponding prompt
def image_generator(directory_path):
    with open(os.path.join(directory_path, "edit_sessions.json"), 'r') as f:
        instructions = json.load(f)

    # Collect all turns into a list
    all_turns = []
    for id, turns in instructions.items():
        for turn in turns:
            all_turns.append((id, turn))

    # Shuffle the list of turns
    random.shuffle(all_turns)

    for id, turn in all_turns:
        folder_path = os.path.join(directory_path, id)
        if os.path.isdir(folder_path):
            original_img = turn["input"]
            mask_img = turn["mask"]
            edited_img = turn["output"]
            prompt = turn["instruction"]
            turn = re.search(r'output(\d+)\.png', edited_img).group(1)

            img_array_original = get_img_array(folder_path, original_img)
            img_array_mask = get_img_array(folder_path, mask_img)
            img_array_edit = get_img_array(folder_path, edited_img)

            yield id, img_array_original, img_array_mask, img_array_edit, prompt, turn

# Create the generator
image_gen = image_generator(directory_path)

In [4]:
# save what ids and turns the annotator has already labeled
annotator_labeled = set()
# save how often turn was already labeled (we need 3)
three_times_labeled = {}

# load existing progress for the annotator
def load_progress(annotator):
    if not os.path.exists("output.json"):
        return set()

    global annotator_labeled, three_times_labeled
    user_progress = set()

    with open("output.json", 'r') as f:
        for line in f:
            entry = json.loads(line)
            if entry.get("annotator") == annotator:
                user_progress.add(entry["id"])
                annotator_labeled.add((entry["id"], entry["turn"]))

            key = (entry["id"], entry["turn"])
            if key in three_times_labeled:
                three_times_labeled[key] += 1
            else:
                three_times_labeled[key] = 1

    return user_progress

In [5]:
def labeled_by_annotator(id, turn):
    return (id, turn) in annotator_labeled

def labeled_three_times(id, turn):
    return three_times_labeled.get((id, turn), 0) >= 3

In [6]:
# save input
user_progress = {}

def record_input(id, alignment, aesthetics, consistency, overall, annotator, turn):
    # if no name was entered, rating won't be registered
    if annotator:
        output = {
            "id": id,
            "turn": turn,
            "alignment": alignment,
            "aesthetics": aesthetics,
            "consistency": consistency,
            "overall": overall,
            "annotator": annotator
        }
        with open("output.json", 'a') as f:
            json.dump(output, f)
            f.write("\n")

        # Update user progress
        if annotator not in user_progress:
            user_progress[annotator] = set()
        user_progress[annotator].add(id)

        annotator_labeled.add((id, turn))
        key = (id, turn)
        three_times_labeled[key] = three_times_labeled.get(key, 0) + 1

    try:
        id, img_array_original, img_array_mask, img_array_edit, prompt, turn = next(image_gen)
        while labeled_by_annotator(id, turn) or labeled_three_times(id, turn):
            id, img_array_original, img_array_mask, img_array_edit, prompt, turn = next(image_gen)
    except StopIteration:
        progress_message = "No more images!", f"You have labeled **{len(user_progress.get(annotator, []))}** out of 528 potential images."
        return id, None, None, None, progress_message, turn, 5, 5, 5, 5

    # Count the number of labeled images
    labeled_count = len(user_progress.get(annotator, []))
    progress_message = f"You have labeled **{labeled_count}** out of 528 potential images."
    return id, img_array_original, img_array_mask, img_array_edit, prompt, progress_message, turn, 5, 5, 5, 5

In [7]:
# start labeling
def start(annotator):
    global user_progress
    user_progress[annotator] = load_progress(annotator)

    try:
        id, img_array_original, img_array_mask, img_array_edit, prompt, turn = next(image_gen)
        while labeled_by_annotator(id, turn) or labeled_three_times(id, turn):
            id, img_array_original, img_array_mask, img_array_edit, prompt, turn = next(image_gen)
    except StopIteration:
        end_message ="No more images!"
        progress_message = f"You have labeled **{len(user_progress.get(annotator, []))}** out of 528 potential images!"
        return 0, None, None, None, end_message, progress_message, turn

    labeled_count = len(user_progress.get(annotator, []))
    progress_message = f"You have labeled **{labeled_count}** out of 528 potential images."
    return id, img_array_original, img_array_mask, img_array_edit, prompt, progress_message, turn

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown(instruction_beginning)

    annotator = gr.Textbox(label="Nickname", interactive=True)

    start_btn = gr.Button("Start")
    progress_text = gr.Markdown("You have labeled **0** out of 528 potential images.")

    # triplet
    with gr.Row():
        img_block1 = gr.Image(visible=True, width=300, height=300, interactive=False, label="Original Image")
        img_block2 = gr.Image(visible=True, width=300, height=300, interactive=False, label="Mask")
        img_block3 = gr.Image(visible=True, width=300, height=300, interactive=False, label="Edited Image")

    prompt = gr.Textbox(label="Prompt", visible=True, interactive=False)
    img_id = gr.Textbox(visible=False)
    turn = gr.Textbox(visible=False)

    # Drei Sliders für die Bewertung
    with gr.Row():
        slider_alignment = gr.Slider(label="Prompt-Image Alignment", minimum=0, maximum=10, step=1, value=5, info=alignment_info)
        slider_aesthetics = gr.Slider(label="Visual Quality", minimum=0, maximum=10, step=1, value=5, info=quality_info)
        slider_consistency = gr.Slider(label="Consistency", minimum=0, maximum=10, step=1, value=5, info=consistency_info)

    slider_overall = gr.Slider(label="Overall Impression", minimum=0, maximum=10, step=1, value=5, info=overall_info)

    save_and_continue_btn = gr.Button("Save & Continue")

    start_btn.click(
        fn=start,
        inputs=[annotator],
        outputs=[img_id, img_block1, img_block2, img_block3, prompt, progress_text, turn]
    )
    save_and_continue_btn.click(
        fn=record_input,
        inputs=[img_id, slider_alignment, slider_aesthetics, slider_consistency, slider_overall, annotator, turn],
        outputs=[img_id, img_block1, img_block2, img_block3, prompt, progress_text, turn, slider_alignment, slider_aesthetics, slider_consistency, slider_overall]
    )

demo.queue()
demo.launch(share=True, debug=True)

In [24]:
# filter images etc -> only use none "remove", only "add, make"

# if three people already labeled this specific turn, do not show again

# every id and turn max 3
# no same image for the person

# mechanism if everything was already labeled 3 times -> stop